<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/13_2d_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2次元の可視化

* 2次元の可視化は、もとは高次元のデータセットの次元を、2次元へと圧縮することである。
  * 前回の演習をふまえると・・・
  * 2次元のような非常に低次元の空間への次元削減は・・・
  * それほど自明なデータ処理ではなさそうだが・・・

## 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS
from umap import UMAP

np.random.seed(42)

%config InlineBackend.figure_format = "retina"

## データセット
* 今回はdigits dataを題材として使う。
  * 8 x 8ピクセルの手書き数字画像

In [ ]:
digits = datasets.load_digits()

In [ ]:
digits.data.shape

In [ ]:
digits.target.shape

In [ ]:
digits.data[0]

In [ ]:
type(digits.data[0, 0])

In [ ]:
plt.imshow(digits.data[0].reshape(8, -1), cmap="gray");

In [ ]:
digits.target

## ノイズ画像の追加

* 各ピクセルの数値の範囲を調べる。

In [ ]:
print(digits.data.min(), digits.data.max())

* 同じ値の範囲で、ランダムな画像を作ってみる。

In [ ]:
noisy_image = np.random.randint(0, high=17, size=64) * 1.0
print(noisy_image)

In [ ]:
plt.imshow(noisy_image.reshape(8, -1), cmap="gray");

* ノイズ画像を、「0」の画像と同じ枚数、新たなインスタンスとして追加する。

In [ ]:
n_noisy_images = (digits.target == 0).sum()
noisy_images = np.random.randint(0, high=17, size=(n_noisy_images, 64))
noisy_images.shape

In [ ]:
digits.data = np.concatenate([digits.data, noisy_images])

* ノイズ画像のラベルは全て「10」にする。

In [ ]:
digits.target = np.concatenate([digits.target, np.full(n_noisy_images, 10)])

In [ ]:
digits.data.shape

In [ ]:
digits.target.shape

* 元のままのデータセットを別の変数名で読み込んでおく。

In [ ]:
original_digits = datasets.load_digits()

## ヘルパ関数
* 下記も参照。
 * https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py

In [ ]:
def scatter_plot(embedding, target, cmap=plt.get_cmap("tab20"), ax=None):
  for color in np.unique(target):
    indices = (target == color)
    if ax is None:
      plt.scatter(embedding[indices, 0], embedding[indices, 1], label=color, color=cmap(color), s=3, alpha=0.5)
    else:
      ax.scatter(embedding[indices, 0], embedding[indices, 1], label=color, color=cmap(color), s=3, alpha=0.5)

## PCAによる可視化

In [ ]:
pca = PCA(10, random_state=42)
embedding = pca.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

* ノイズ画像なしの場合

In [ ]:
pca = PCA(10, random_state=42)
embedding = pca.fit_transform(original_digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, original_digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

## UMAPによる可視化
* https://umap-learn.readthedocs.io/en/latest/parameters.html


* どんな可視化ツールにも、調整できるパラメータがある。
* UMAPの場合は・・・
  * パラメータ`n_neighbors`を変えると可視化がどう変わるか。
  * パラメータ`min_dist`を変えると可視化がどう変わるか。


### デフォルトの設定で可視化

In [ ]:
%%time
reducer = UMAP(n_jobs=1, random_state=42)
embedding = reducer.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

* ノイズ画像なしの場合

In [ ]:
reducer = UMAP(n_jobs=1, random_state=42)
embedding = reducer.fit_transform(original_digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, original_digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

### `n_neighbors`を変更する
* デフォルトの値は15

In [ ]:
def draw_umap(digits, n_neighbors=15, min_dist=0.1, title=""):
  reducer = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_jobs=1, random_state=42)
  u = reducer.fit_transform(digits.data)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  scatter_plot(u, digits.target, ax=ax)
  plt.setp(ax, xticks=[], yticks=[])
  plt.legend()
  plt.title(title, fontsize=15);

In [ ]:
for n in (5, 10, 20, 50, 100, 200):
  title = f"n_neighbors = {n}"
  print(title)
  draw_umap(digits, n_neighbors=n, title=title)

### `min_dist`を変更する
* デフォルトの値は0.1

In [ ]:
for d in (0.0, 0.1, 0.25, 0.5, 0.8, 0.99):
  title = f"min_dist = {d}"
  print(title)
  draw_umap(digits, min_dist=d, title=title)

## t-SNEによる可視化
* https://scikit-learn.org/stable/auto_examples/manifold/plot_t_sne_perplexity.html
 * パラメータ`perplexity`を変えると可視化がどう変わるか。

### デフォルトの設定で可視化

In [ ]:
%%time
reducer = TSNE(random_state=42)
embedding = reducer.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

* ノイズ画像なしの場合

In [ ]:
%%time
reducer = TSNE(random_state=42)
embedding = reducer.fit_transform(original_digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, original_digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

### `perplexity`を変更する

In [ ]:
def draw_tsne(digits, perplexity=30.0, title=""):
  reducer = TSNE(perplexity=perplexity, random_state=42)
  u = reducer.fit_transform(digits.data)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  scatter_plot(u, digits.target, ax=ax)
  plt.setp(ax, xticks=[], yticks=[])
  plt.legend()
  plt.title(title, fontsize=15);

In [ ]:
for p in (2, 5, 10, 20, 50, 100):
  title = f"perplexity = {p}"
  print(title)
  draw_tsne(digits, perplexity=p, title=title)

## MDSによる可視化
* https://scikit-learn.org/stable/modules/manifold.html#multidimensional-scaling

### デフォルトの設定で可視化

* やや時間がかかる（3分間弱）。

In [ ]:
%%time
reducer = MDS(normalized_stress="auto", random_state=42)
embedding = reducer.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

* ノイズ画像なしの場合

In [ ]:
%%time
reducer = MDS(normalized_stress="auto", random_state=42)
embedding = reducer.fit_transform(original_digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, original_digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

* epsを変更してみたが、今回のデータセットでは、ほとんど変化が見られなかった。

## 考察
* それぞれの可視化ツールを、デフォルトの設定で使っても構わないか？
* 2次元の可視化において、遠いものは遠いと言っていいか？
* 2次元の可視化において、近いものは近いと言っていいか？
* digits dataに関して結論して構わないことは、何か？ 例えば・・・
  * 「2」と「7」の位置関係について、何か言えるか？
  * 「4」と「6」の位置関係について、何か言えるか？
* 新たに追加したノイズ画像は**明らかに異質な画像群**だが・・・
  * 各手法は、そのように可視化してくれていたか？

## 助言
* 初めに結論ありきの、"自分が見たいものだけを見る可視化"にならないよう、注意しよう。
* 複数の可視化手法を比較するようにしよう。

* 参考
  * https://distill.pub/2016/misread-tsne/
  * https://simplystatistics.org/posts/2024-12-23-biologists-stop-including-umap-plots-in-your-papers/
  * https://www.arxiv.org/abs/2506.08725

# プランナー課題１３
* 追加するnoisyな画像の作り方を、以下のように変更する。
  * 元のデータセットから、ランダムに2枚の画像を選ぶ。
  * 選ばれた一方の画像の上半分と、他方の画像の下半分をくっつける。
* このような画像を追加した後に・・・
* 上と同じように、可視化手法のパラメータを変更すると、可視化結果がどのように変わるか、観察しよう。

### ランダムに選んだ2枚の画像の上下を結合する関数

In [ ]:
def generate_noisy_image():
  while True:
    idx1, idx2 = np.random.randint(len(digits.target), size=2)
    # 違う数字の画像であることの確認
    if digits.target[idx1] != digits.target[idx2]:
      break
  noisy_image = np.zeros(64)
  noisy_image[:32] = digits.data[idx1, :32]
  noisy_image[32:] = digits.data[idx2, 32:]
  return noisy_image

* どんな画像が出来上がるかを確認する。

In [ ]:
noisy_image = generate_noisy_image()
plt.imshow(noisy_image.reshape(8, -1), cmap="gray");

* 元の画像群と区別しにくそうであることが分かる。

* このようなノイズ画像を追加したデータセットを作る。

In [ ]:
digits = datasets.load_digits()

In [ ]:
n_noisy_images = (digits.target == 0).sum()
noisy_images = np.zeros((n_noisy_images, 64))
for i in range(n_noisy_images):
  noisy_images[i, :] = generate_noisy_image()

In [ ]:
digits.data = np.concatenate([digits.data, noisy_images])

In [ ]:
digits.target = np.concatenate([digits.target, np.full(n_noisy_images, 10)])

In [ ]:
digits.data.shape

In [ ]:
digits.target.shape

* 元のままのデータセットを別の変数名で読み込んでおく。

In [ ]:
original_digits = datasets.load_digits()

## PCAによる可視化

In [ ]:
pca = PCA(10, random_state=42)
embedding = pca.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

In [ ]:
pca = PCA(10, random_state=42)
embedding = pca.fit_transform(original_digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, original_digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend();

* 以下、いろいろ可視化してみよう。
  * 追加する画像の数を増やすとどうなるだろうか。